<a href="https://colab.research.google.com/github/chaotingchong-crypto/programming-language/blob/main/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import gradio as gr
import pandas as pd
import datetime
import gspread
from google.colab import auth
from google.auth import default


In [2]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1jNgcmfmYeYiA8GirXALrm3D5FbvPjkULxGp76NcCE74/edit?hl=zh-tw&gid=0#gid=0"
WORKSHEET_NAME = "Sheet1"

REQUIRED_COLUMNS = ["日期", "品項", "數量", "金額"]

_auth_done = False
_gc = None
_ws = None

In [ ]:
def _ensure_auth_and_ws():
    global _auth_done, _gc, _ws
    if not _auth_done:
        auth.authenticate_user()
        creds, _ = default()
        _gc = gspread.authorize(creds)
        _auth_done = True
    if _ws is None:
        gs = _gc.open_by_url(SHEET_URL)
        _ws = gs.worksheet(WORKSHEET_NAME)
        _ensure_headers()
    return _ws

def _ensure_headers():
    rows = _ws.get_all_values()
    if not rows:
        _ws.append_row(REQUIRED_COLUMNS, value_input_option="USER_ENTERED")
        return
    header = rows[0]
    if header != REQUIRED_COLUMNS:
        existing = {h: idx for idx, h in enumerate(header)}
        _ws.update('1:1', [REQUIRED_COLUMNS])
        if len(rows) > 1:
            new_rows = []
            for r in rows[1:]:
                mapping = {col: (r[existing[col]] if col in existing and existing[col] < len(r) else "")
                           for col in REQUIRED_COLUMNS}
                new_rows.append([mapping[c] for c in REQUIRED_COLUMNS])
            _ws.resize(rows=1)
            _ws.append_rows(new_rows, value_input_option="USER_ENTERED")

def _read_df():
    ws = _ensure_auth_and_ws()
    values = ws.get_all_values()
    if not values:
        return pd.DataFrame(columns=REQUIRED_COLUMNS)
    df = pd.DataFrame(values[1:], columns=values[0])
    if "數量" in df.columns:
        df["數量"] = pd.to_numeric(df["數量"], errors="coerce").fillna(0).astype(int)
    if "金額" in df.columns:
        df["金額"] = pd.to_numeric(df["金額"], errors="coerce").fillna(0.0)
    return df

def add_expense(date_str, item, qty, amount):
    try:
        if not date_str:
            date_str = datetime.date.today().strftime("%Y-%m-%d")
        item = (item or "未填").strip()

        try:
            qty_val = int(qty)
        except:
            return "數量格式錯誤，請輸入整數", None, None

        try:
            amount_val = float(amount)
        except:
            return "金額格式錯誤，請輸入數字", None, None

        ws = _ensure_auth_and_ws()
        ws.append_row(
            [date_str, item, qty_val, amount_val],
            value_input_option="USER_ENTERED"
        )
        msg = f"✅ 已新增：{date_str}｜{item}｜{qty_val}｜{amount_val}"
        df = _read_df()
        total = float(df["金額"].sum()) if not df.empty else 0.0
        return msg, total, df
    except Exception as e:
        return f"❌ 新增失敗：{e}", None, None

def refresh_summary():
    try:
        df = _read_df()
        if df.empty:
            return "目前沒有資料", 0.0, pd.DataFrame()
        total = float(df["金額"].sum())
        return "✅ 已更新彙總", total, df
    except Exception as e:
        return f"❌ 讀取失敗：{e}", 0.0, pd.DataFrame()

with gr.Blocks(title="日常支出紀錄（四欄版）") as demo:
    gr.Markdown("## 🧾 日常支出紀錄（Gradio）\n- 新增支出後自動寫回 Google Sheet\n- 讀取總額與原始資料\n- 欄位：日期、品項、數量、金額")

    with gr.Tab("➕ 新增支出"):
        with gr.Row():
            date_in = gr.Textbox(label="日期 YYYY-MM-DD", value=datetime.date.today().strftime("%Y-%m-%d"))
            item_in = gr.Textbox(label="品項", placeholder="如 咖啡 / 便當 / 車票")
        with gr.Row():
            qty_in = gr.Textbox(label="數量", placeholder="整數")
            amt_in = gr.Textbox(label="金額", placeholder="數字")
        add_btn = gr.Button("新增到工作表")

        add_msg = gr.Markdown()
        total_out = gr.Number(label="目前總額", interactive=False)
        data_df = gr.Dataframe(label="目前資料", interactive=False)

        add_btn.click(
            fn=add_expense,
            inputs=[date_in, item_in, qty_in, amt_in],
            outputs=[add_msg, total_out, data_df]
        )

    with gr.Tab("📒 檢視 / 更新"):
        refresh_btn = gr.Button("讀取最新資料")
        msg2 = gr.Markdown()
        total2 = gr.Number(label="總額", interactive=False)
        raw_preview = gr.Dataframe(label="全部資料", interactive=False)

        refresh_btn.click(
            fn=refresh_summary,
            inputs=[],
            outputs=[msg2, total2, raw_preview]
        )

# 啟動介面
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://180ce83d5fd3bb8eec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
